In [0]:
from pyspark.sql.functions import current_timestamp

def add_ingestion_date(input_df):
    output_df = input_df.withColumn('ingestion_date', current_timestamp())
    return output_df

def position_partition_column_last(df, partition_column):
    column_list = df.schema.names
    column_list.remove(partition_column)
    column_list.append(partition_column)
    return df.select(column_list)

def incremental_load(df, database, table, partition_column):
    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    df = position_partition_column_last(df, partition_column)

    if (spark._jsparkSession.catalog().tableExists(f"{database}.{table}")):
        df.write.mode('overwrite')\
            .insertInto(f"{database}.{table}")
    else:
        df.write.mode('append')\
            .partitionBy(partition_column)\
            .format('parquet')\
            .saveAsTable(f"{database}.{table}")